In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm -r sample_data/

In [ ]:
output_dir = 'drive/MyDrive/BigData-finalproject/Analysis_Outputs/'

# setup spark

In [ ]:
#@title spark install { form-width: "20%" }
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

!pip install pyspark


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [ ]:
#@title spark setup { form-width: "20%" }

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"


import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.driver.memory", "10g").getOrCreate()
sc = spark.sparkContext

In [ ]:
!rm spark-2.4.4-bin-hadoop2.7.tgz

# read data

In [ ]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [ ]:
df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('/content/drive/MyDrive/BigData-finalproject/complete_data/complete_clean_data.csv').persist()


In [ ]:
df.show()

+----+-----+----------+---------+-----------------+-----------+-------------------------------+---------------+------+--------------+-----------+---------------+-------------+----+--------------------+---------+-------------------+-------+--------+-------+--------+---------+--------+-----------------+-------+--------+
|Year|Month|DayofMonth|DayOfWeek|Reporting_Airline|Tail_Number|Flight_Number_Reporting_Airline|OriginAirportID|Origin|OriginCityName|OriginState|OriginStateName|DestAirportID|Dest|        DestCityName|DestState|      DestStateName|DepTime|DepDelay|ArrTime|ArrDelay|Cancelled|Diverted|ActualElapsedTime|Flights|Distance|
+----+-----+----------+---------+-----------------+-----------+-------------------------------+---------------+------+--------------+-----------+---------------+-------------+----+--------------------+---------+-------------------+-------+--------+-------+--------+---------+--------+-----------------+-------+--------+
|1995|    1|        26|        4|       

In [ ]:
N= df.count()
N

82660244

In [ ]:
df.groupBy('Year').count().show()

+----+-------+
|Year|  count|
+----+-------+
|1995|5225015|
|1996|5209305|
|1997|5301975|
|1998|5227026|
|1999|5359994|
|2000|5481278|
|2001|5723646|
|2002|5197835|
|2003|6375599|
|2004|6987150|
|2005|6992750|
|2006|7003613|
|2007| 907251|
|2019|7268232|
|2020|4399575|
+----+-------+



In [ ]:
df.groupBy('OriginCityName').count().show()

+--------------------+------+
|      OriginCityName| count|
+--------------------+------+
|     Gainesville, FL| 21613|
|         Ontario, CA|468249|
|        Richmond, VA|232123|
|          Tucson, AZ|260911|
|    Myrtle Beach, SC| 67063|
|         Medford, OR| 38649|
|    Palm Springs, CA|105364|
|         Durango, CO| 13673|
|          Mobile, AL| 60280|
|  Corpus Christi, TX| 74722|
|        Columbus, GA| 10353|
|      Huntsville, AL| 97359|
|       Pensacola, FL|102442|
|      Fort Myers, FL|275822|
|Pasco/Kennewick/R...| 29456|
|        San Juan, PR|322885|
|  Montrose/Delta, CO| 15521|
|Harlingen/San Ben...| 67020|
|            Reno, NV|353863|
|        Columbus, OH|522365|
+--------------------+------+
only showing top 20 rows



In [ ]:
df.groupBy('DestCityName').count().show()

In [ ]:
len(df.columns)

26

In [ ]:
df.columns

['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'Reporting_Airline',
 'Tail_Number',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateName',
 'DestAirportID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateName',
 'DepTime',
 'DepDelay',
 'ArrTime',
 'ArrDelay',
 'Cancelled',
 'Diverted',
 'ActualElapsedTime',
 'Flights',
 'Distance']

# REQUIREMENT 1: best month, day of month/ of week to minimize delay.

1- group by:
>* month
>* day of month
>* day of week

2- aggregate the average of departure delay - and arrival delay 

3- sort ascendingly by delay

In [ ]:
df1 = df.select(['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
  'DepDelay',
  'ArrDelay',
  'Cancelled'
  ]
 )

df1= df1.persist()
df1.show()

[Row(Year=1995, Month=1, DayofMonth=26, DayOfWeek=4, DepDelay=13.0, ArrDelay=8.0, Cancelled=0.0)]

In [ ]:
# select non-cancelled flights
df1= df1.filter('Cancelled != 1.0')
month = df1.groupBy('Month').agg(F.avg('DepDelay'),F.avg('ArrDelay'))
monthday  = df1.groupBy('DayofMonth').agg(F.avg('DepDelay').alias('DepDelay'),F.avg('ArrDelay').alias('ArrDelay'))
weekday  = df1.groupBy('DayOfWeek').agg(F.avg('DepDelay').alias('DepDelay'),F.avg('ArrDelay').alias('ArrDelay'))



In [ ]:
weekday.show()

+---------+------------------+-----------------+
|DayOfWeek|          DepDelay|         ArrDelay|
+---------+------------------+-----------------+
|        1| 8.250621514451984|6.227595693902848|
|        6| 6.956175525616974|3.358325510591456|
|        3| 7.540787986294303|6.069135502788045|
|        5|10.352147414655057|8.870210670117741|
|        4| 9.472349158393014| 8.28423163986844|
|        7|  8.67867908548572| 6.00716189930581|
|        2| 6.725080962996344|4.865935994234928|
+---------+------------------+-----------------+



In [ ]:
# save the analysis outputs 
month.toPandas().to_csv(output_dir+'req1/'+'month_delay.csv', index=False)
monthday.toPandas().to_csv(output_dir+'req1/'+'monthday_delay.csv', index=False)
weekday.toPandas().to_csv(output_dir+'req1/'+'weekday_delay.csv', index=False)


In [ ]:
df1 = df1.unpersist()
df1.show()

+----+-----+----------+---------+--------+--------+---------+
|Year|Month|DayofMonth|DayOfWeek|DepDelay|ArrDelay|Cancelled|
+----+-----+----------+---------+--------+--------+---------+
|1995|    1|        26|        4|    13.0|     8.0|      0.0|
|1995|    1|        27|        5|     2.0|     2.0|      0.0|
|1995|    1|        28|        6|    -1.0|     2.0|      0.0|
|1995|    1|        29|        7|     0.0|    -5.0|      0.0|
|1995|    1|        30|        1|    40.0|    43.0|      0.0|
|1995|    1|        31|        2|    12.0|    22.0|      0.0|
|1995|    1|         1|        7|    -3.0|   -19.0|      0.0|
|1995|    1|         2|        1|     0.0|    41.0|      0.0|
|1995|    1|         3|        2|    -1.0|   -11.0|      0.0|
|1995|    1|         4|        3|    -2.0|    -3.0|      0.0|
|1995|    1|         5|        4|    -3.0|    -7.0|      0.0|
|1995|    1|         6|        5|    14.0|     6.0|      0.0|
|1995|    1|         7|        6|    65.0|    50.0|      0.0|
|1995|  

In [ ]:
del df1
del month
del monthday
del weekday

# REQUIREMENT 2:  2020 travel patterns vs prior years (effect of covid19 and lockdown)

1- travel frequency (number of flights) across years && across the months of 2020 and 2019 
> * group by (year, month) should yeald 15yrs* 12 months  
> * count the rows (flights)
> * aggregate the delays 

2- flight calcellation rate across years && across the months of 2019 and 2020
> * group by (year, month) should yeald 15yrs* 12 months  
> * count the rows (flights)
> * aggregate the delays

In [ ]:
# cancellation records can be found in a separate csv file
cancellation = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('/content/drive/MyDrive/BigData-finalproject/complete_data/cancellation.csv').persist()


In [ ]:
df2_cancel = cancellation.select(['Year',
 'Month',
  'Cancelled'])

month_years_cancel = df2_cancel.groupBy('Year', 'Month').agg(F.sum('Cancelled').alias('TotalCancelled'))

just_years_cancel=  month_years_cancel.groupBy('Year').agg(F.sum('TotalCancelled').alias('TotalCancelled'))

df_19_20_cancel= month_years_cancel.filter(month_years_cancel.Year >= 2019)

In [ ]:
month_years_cancel.show()

+----+-----+--------------+
|Year|Month|TotalCancelled|
+----+-----+--------------+
|1996|   11|        9547.0|
|1996|   10|        7947.0|
|1996|   12|       12330.0|
|1996|    1|       29594.0|
|1996|    2|       14586.0|
|1996|    4|        5522.0|
|1996|    5|        7043.0|
|1996|    3|        8923.0|
|1996|    9|        7720.0|
|1996|    6|        8023.0|
|1996|    7|        9350.0|
|1996|    8|        7948.0|
|1997|   11|        6727.0|
|1997|   10|        7533.0|
|1997|   12|        8215.0|
|1997|    1|       18142.0|
|1997|    3|        8347.0|
|1997|    2|        8613.0|
|1997|    5|        5179.0|
|1997|    4|        7054.0|
+----+-----+--------------+
only showing top 20 rows



In [ ]:
just_years_cancel.show()

+----+--------------+
|Year|TotalCancelled|
+----+--------------+
|2003|      101469.0|
|2007|       19044.0|
|2006|      121932.0|
|1997|       97763.0|
|2019|      134925.0|
|2004|      127756.0|
|1996|      128533.0|
|1998|      144509.0|
|2020|      281034.0|
|2001|      231198.0|
|2005|      133729.0|
|2000|      187490.0|
|1999|      154311.0|
|2002|       65143.0|
+----+--------------+



In [ ]:
df_19_20_cancel.show()

+----+-----+--------------+
|Year|Month|TotalCancelled|
+----+-----+--------------+
|2019|    8|       11298.0|
|2019|    1|       16726.0|
|2020|    6|         813.0|
|2020|    4|      130078.0|
|2020|    1|        6928.0|
|2019|   11|        4446.0|
|2019|    2|       15255.0|
|2020|    8|        3921.0|
|2019|    7|       12928.0|
|2019|    5|       13012.0|
|2020|    3|      110187.0|
|2020|   10|        1860.0|
|2020|    5|       11613.0|
|2019|    3|       12564.0|
|2019|    4|       14488.0|
|2019|    9|       10016.0|
|2019|    6|       13227.0|
|2020|   12|        3850.0|
|2020|    7|        2813.0|
|2019|   10|        5172.0|
+----+-----+--------------+
only showing top 20 rows



In [ ]:
df2 = df.select(['Year',
 'Month',
  'DepDelay',
  'ArrDelay']).persist()

month_years_df = df2.groupBy('Year', 'Month').agg(F.count(F.lit(1)).alias("NumFlights"),\
                                                 F.avg('DepDelay').alias('AvgDepDelay'),  \
                                                F.avg('ArrDelay').alias('AvgArrDelay'))

just_years_df =  month_years_df.groupBy('Year').agg(F.sum('NumFlights').alias("NumFlights"),\
                                                 F.avg('AvgDepDelay').alias('AvgDepDelay'),  \
                                                F.avg('AvgArrDelay').alias('AvgArrDelay'))

df_19_20= month_years_df.filter(month_years_df.Year >= 2019)

In [ ]:
# now join the delay dataframes with the cancellation dataframes
month_years_df = month_years_df.join(month_years_cancel, ['Year','Month'])

just_years_df = just_years_df.join(just_years_cancel, 'Year')

df_19_20=df_19_20.join(df_19_20_cancel, ['Year','Month'])


In [ ]:
month_years_df.show()

+----+-----+----------+------------------+------------------+
|Year|Month|NumFlights|       AvgDepDelay|       AvgArrDelay|
+----+-----+----------+------------------+------------------+
|1995|   11|    419996| 8.950294764712044|7.2848931894589475|
|1995|    1|    449322| 9.872033864355629|  8.92083850779619|
|1995|   10|    442435| 6.463989060539967| 5.006541073830054|
|1995|   12|    427851|14.251825986149385|13.923333122979729|
|1995|    2|    407602| 7.625693691395037| 6.123757979597745|
|1995|    4|    434493| 6.855067860702014| 5.721012766603835|
|1995|    3|    453411| 7.713416745513452| 6.202434435865032|
|1995|    6|    432696| 9.122772107900234|  8.93082672361196|
|1995|    5|    441102| 6.738894858785496| 5.916182651631596|
|1995|    8|    448822| 8.265686619639856| 6.744921149141531|
|1995|    7|    442970| 8.127069553242883| 6.732230173600921|
|1995|    9|    424315| 4.831884331216195|2.7882987874574314|
|1996|    1|    415679|14.928670921552447|15.667909612946529|
|1996|  

In [ ]:
just_years_df.show()

+----+----------+------------------+------------------+
|Year|NumFlights|       AvgDepDelay|       AvgArrDelay|
+----+----------+------------------+------------------+
|2003|   6375599|5.2188466488959575|3.5947442261521334|
|2007|    907251| 9.185091039705124| 7.718414021139613|
|2006|   7003613|10.032893450323556| 8.648847562289042|
|1997|   5301975| 8.198671638971275| 7.503388950953422|
|2019|   7268232|10.827840413792517| 5.401193242336937|
|2004|   6987150| 7.842819196299234| 6.490689763857973|
|1996|   5209305|  9.94386397650566| 9.685234379159207|
|1998|   5227026|  8.94998688012192| 7.568089706725664|
|2020|   4399575|1.0701106255313064|-6.089619917108048|
|1995|   5225015| 8.234885787012685| 7.024605880131248|
|2001|   5723646| 8.010570929281338| 5.347377457131327|
|2005|   6992750| 8.615297284715703| 7.146218020142608|
|2000|   5481278| 11.19853490240074| 10.46565140873958|
|1999|   5359994| 9.283238550649385|  8.24644253867114|
|2002|   5197835|5.4683726777893655| 3.155397082

In [ ]:
df_19_20.show()

+----+-----+----------+-------------------+--------------------+
|Year|Month|NumFlights|        AvgDepDelay|         AvgArrDelay|
+----+-----+----------+-------------------+--------------------+
|2019|    8|    645351| 12.576317383873272|   7.497318513491108|
|2019|    2|    516314| 13.259266647815089|   8.729741978718376|
|2019|   12|    618612| 12.288182253173233|   6.119191027655461|
|2020|   11|    361799| -0.320122498956603|  -6.543011451109594|
|2020|    9|    321092|-0.5371326597984378|  -7.465813536307351|
|2020|    7|    349256| 0.6222598895938796|  -5.012907437524338|
|2019|    4|    596020|  10.56446260192611|   4.853110633871347|
|2019|    9|    594716|  6.370699628057762| 0.21388864600918758|
|2020|    3|    537244| 1.4674766028098964|  -6.355562835508633|
|2020|    4|    183006| -3.272176868517972| -13.626482191840704|
|2019|    5|    621339| 12.000135191900075|  7.0522709825071335|
|2020|    2|    568076|  7.357084263373211|-0.19684161978326845|
|2020|   10|    349709| 0

In [ ]:
# save the analysis outputs 
month_years_df.toPandas().to_csv(output_dir+'req2/'+'year_months.csv', index=False)
just_years_df.toPandas().to_csv(output_dir+'req2/'+'just_years.csv', index=False)
df_19_20.toPandas().to_csv(output_dir+'req2/'+'just_19_20.csv', index=False)


In [ ]:
del df2
del month_years_df
del just_years_df
del df_19_20

# REQUIREMENT 3: flight patterns pre-and-post 9/11 
 this is the exact same process as the previous requirement but we focus on the year 2001

In [ ]:
df3 = df.filter('Year ==2001')
df3_cancel = cancellation.filter('Year ==2001')

In [ ]:
delays_2001 = df3.groupBy('Month', 'DayOfMonth').agg(F.count(F.lit(1)).alias("NumFlights"),\
                                                 F.avg('DepDelay').alias('AvgDepDelay'),  \
                                                F.avg('ArrDelay').alias('AvgArrDelay'))

cancel_2001 = df3_cancel.groupBy('Month', 'DayOfMonth').agg(F.sum('Cancelled').alias('TotalCancelled'))

df_2001 = delays_2001.join(cancel_2001, ['Month', 'DayOfMonth']).withColumnRenamed('Month', 'MonthOf2001')
df_2001.show()

+-----+----------+----------+------------------+--------------------+--------------+
|Month|DayOfMonth|NumFlights|       AvgDepDelay|         AvgArrDelay|TotalCancelled|
+-----+----------+----------+------------------+--------------------+--------------+
|   10|         2|     14405|0.8705310656022215|   -6.66872613675807|         367.0|
|   11|        23|     11035|0.5352967829632986| -5.5659265971907566|          50.0|
|   12|        30|     13230| 9.893801965230537|   6.163643235071807|          76.0|
|   12|        19|     14472| 8.765478164731896|    5.90616362631288|         131.0|
|    1|        25|     17045| 5.391551774713992|  2.4698151950718685|         464.0|
|   12|         7|     14326| 2.533924333379869|-0.04907161803713528|          81.0|
|   10|        15|     14416| 6.024694783573807|  0.9358351831298557|         127.0|
|   10|        24|     13993|13.163867648109768|  13.695847923961981|         424.0|
|   12|        17|     14358| 5.698774202535172|    4.16220922134

In [ ]:
df_sept = df_2001.filter('MonthOf2001== 9')

In [ ]:
df_2001.toPandas().to_csv(output_dir+'req3/'+'whole_year_2001.csv', index=False)
df_sept.toPandas().to_csv(output_dir+'req3/'+'sept_2001.csv', index=False)


# REQUIREMENT 4: delay patterns per airport - the airports associated with the highest departure delays

1-group by origin airport ID, origin city, and state


2- agregate the average departure delay

In [ ]:
df4= df.select(['OriginAirportID',
 'Origin',
 'OriginCityName',
 'OriginStateName',
 'DepDelay'])

In [ ]:
airport_delay = df4.groupBy('OriginAirportID').agg(F.collect_set('Origin').alias('Origin'),\
                                                   F.collect_set('OriginCityName').alias('OriginCityName'),\
                                                   F.collect_set('OriginStateName').alias('OriginStateName'),\
                                                   F.avg('DepDelay').alias('AvgDepDelay')
                                                   )

airport_delay.show()


+---------------+------+--------------------+---------------+------------------+
|OriginAirportID|Origin|      OriginCityName|OriginStateName|       AvgDepDelay|
+---------------+------+--------------------+---------------+------------------+
|          10135| [ABE]|[Allentown/Bethle...| [Pennsylvania]| 8.851955044081029|
|          10136| [ABI]|       [Abilene, TX]|        [Texas]|3.1360820655136528|
|          10140| [ABQ]|   [Albuquerque, NM]|   [New Mexico]| 7.074627021018383|
|          10141| [ABR]|      [Aberdeen, SD]| [South Dakota]|12.567988668555241|
|          10155| [ACT]|          [Waco, TX]|        [Texas]| 0.870302781062047|
|          10165| [ADK]|   [Adak Island, AK]|       [Alaska]| 22.51685393258427|
|          10208| [AGS]|       [Augusta, GA]|      [Georgia]| 8.726899448345035|
|          10257| [ALB]|        [Albany, NY]|     [New York]| 6.773269664184498|
|          10268| [ALO]|      [Waterloo, IA]|         [Iowa]| 5.451757188498402|
|          10272| [ALS]|    

In [ ]:
airport_delay.toPandas().to_csv(output_dir+'req4/'+'airport_delays.csv')

In [ ]:
del df4
del airport_delay

# REQUIREMENT 5: How does the number of people flying between different locations change over time?

- Two analyses here; one for the states pair, and another for the origin state change with the years.
Done by concatenating, grouping, counting, and ordering.

In [ ]:
year_state = df.groupBy('OriginState', 'Year').count().orderBy('OriginState', 'Year')
year_state.show()

+-----------+----+-----+
|OriginState|Year|count|
+-----------+----+-----+
|         AK|1995|40339|
|         AK|1996|41601|
|         AK|1997|40563|
|         AK|1998|42092|
|         AK|1999|41454|
|         AK|2000|41014|
|         AK|2001|42565|
|         AK|2002|42996|
|         AK|2003|42240|
|         AK|2004|41595|
|         AK|2005|41674|
|         AK|2006|40638|
|         AK|2007| 2956|
|         AK|2019|39101|
|         AK|2020|26113|
|         AL|1995|32427|
|         AL|1996|30500|
|         AL|1997|31066|
|         AL|1998|30905|
|         AL|1999|31557|
+-----------+----+-----+
only showing top 20 rows



In [ ]:
year_2states = df.select(F.concat_ws('-', df['OriginState'], df['DestState']).alias('orig-dst'), 'Year').groupBy('orig-dst', 'Year').count().orderBy('orig-dst', 'Year')
year_2states.show()

+--------+----+-----+
|orig-dst|Year|count|
+--------+----+-----+
|   AK-AK|1995|27867|
|   AK-AK|1996|27972|
|   AK-AK|1997|27705|
|   AK-AK|1998|28872|
|   AK-AK|1999|27780|
|   AK-AK|2000|27212|
|   AK-AK|2001|28197|
|   AK-AK|2002|28917|
|   AK-AK|2003|27990|
|   AK-AK|2004|26681|
|   AK-AK|2005|26204|
|   AK-AK|2006|25669|
|   AK-AK|2007| 1997|
|   AK-AK|2019|22342|
|   AK-AK|2020|15917|
|   AK-AZ|1998|  126|
|   AK-AZ|1999|  145|
|   AK-AZ|2000|   96|
|   AK-AZ|2001|   95|
|   AK-AZ|2004|  212|
+--------+----+-----+
only showing top 20 rows



In [ ]:
year_2states.where(year_2states['orig-dst'] == 'MA-NY').show()

+--------+----+-----+
|orig-dst|Year|count|
+--------+----+-----+
|   MA-NY|1995| 9375|
|   MA-NY|1996| 8717|
|   MA-NY|1997| 9106|
|   MA-NY|1998| 8092|
|   MA-NY|1999| 8272|
|   MA-NY|2000|11535|
|   MA-NY|2001|20727|
|   MA-NY|2002|17925|
|   MA-NY|2003|17375|
|   MA-NY|2004|17689|
|   MA-NY|2005|17459|
|   MA-NY|2006|21208|
|   MA-NY|2007| 2465|
|   MA-NY|2019|19595|
|   MA-NY|2020| 7556|
+--------+----+-----+



In [ ]:
year_2states.toPandas().to_csv(output_dir+'req5/'+'year_2states.csv')
year_state.toPandas().to_csv(output_dir+'req5/'+'year_state.csv')

# REQUIREMENT 6: cascading failures as delays in one airport create delays in others.
- Took forever

In [ ]:
#Takes too much time

In [ ]:
req6 = df.select('OriginAirportID', 'OriginState', F.concat_ws('-', df.Year, df.Month, df.DayofMonth).alias('year-month-day'), 'DepDelay', 'Deptime')

In [ ]:
req6.show()

+---------------+-----------+--------------+--------+-------+
|OriginAirportID|OriginState|year-month-day|DepDelay|Deptime|
+---------------+-----------+--------------+--------+-------+
|          13303|         FL|     1995-1-26|    13.0| 1718.0|
|          13303|         FL|     1995-1-27|     2.0| 1707.0|
|          13303|         FL|     1995-1-28|    -1.0| 1704.0|
|          13303|         FL|     1995-1-29|     0.0| 1705.0|
|          13303|         FL|     1995-1-30|    40.0| 1745.0|
|          13303|         FL|     1995-1-31|    12.0| 1717.0|
|          13303|         FL|      1995-1-1|    -3.0| 1132.0|
|          13303|         FL|      1995-1-2|     0.0| 1135.0|
|          13303|         FL|      1995-1-3|    -1.0| 1134.0|
|          13303|         FL|      1995-1-4|    -2.0| 1133.0|
|          13303|         FL|      1995-1-5|    -3.0| 1132.0|
|          13303|         FL|      1995-1-6|    14.0| 1149.0|
|          13303|         FL|      1995-1-7|    65.0| 1240.0|
|       

In [ ]:
rc = req6.select("*")
rc = rc.where(rc.OriginAirportID != 13303)
rc.show()

+---------------+-----------+--------------+--------+-------+
|OriginAirportID|OriginState|year-month-day|DepDelay|Deptime|
+---------------+-----------+--------------+--------+-------+
|          14492|         NC|      1995-1-1|    -1.0| 1014.0|
|          14492|         NC|      1995-1-2|     2.0| 1017.0|
|          14492|         NC|      1995-1-3|    -3.0| 1012.0|
|          14492|         NC|      1995-1-4|    15.0| 1030.0|
|          14492|         NC|      1995-1-5|    12.0| 1027.0|
|          14492|         NC|      1995-1-6|     0.0| 1015.0|
|          14492|         NC|      1995-1-7|    28.0| 1043.0|
|          14492|         NC|      1995-1-8|    -3.0| 1012.0|
|          14492|         NC|      1995-1-9|    -2.0| 1013.0|
|          14492|         NC|     1995-1-10|    -1.0| 1014.0|
|          14492|         NC|     1995-1-11|    -3.0| 1012.0|
|          14492|         NC|     1995-1-12|    81.0| 1136.0|
|          14492|         NC|     1995-1-13|    -1.0| 1014.0|
|       

In [ ]:
r = req6.select("*")
r = r.withColumnRenamed('DepDelay', 'refDelay').withColumnRenamed('Deptime', 'refDep').where(r.OriginAirportID == 13303)
r.show()

+---------------+-----------+--------------+--------+------+
|OriginAirportID|OriginState|year-month-day|refDelay|refDep|
+---------------+-----------+--------------+--------+------+
|          13303|         FL|     1995-1-26|    13.0|1718.0|
|          13303|         FL|     1995-1-27|     2.0|1707.0|
|          13303|         FL|     1995-1-28|    -1.0|1704.0|
|          13303|         FL|     1995-1-29|     0.0|1705.0|
|          13303|         FL|     1995-1-30|    40.0|1745.0|
|          13303|         FL|     1995-1-31|    12.0|1717.0|
|          13303|         FL|      1995-1-1|    -3.0|1132.0|
|          13303|         FL|      1995-1-2|     0.0|1135.0|
|          13303|         FL|      1995-1-3|    -1.0|1134.0|
|          13303|         FL|      1995-1-4|    -2.0|1133.0|
|          13303|         FL|      1995-1-5|    -3.0|1132.0|
|          13303|         FL|      1995-1-6|    14.0|1149.0|
|          13303|         FL|      1995-1-7|    65.0|1240.0|
|          13303|       

In [ ]:
joint= rc.join(r.select('year-month-day', 'refDelay', 'refDep'), ['year-month-day'])
joint.show()

In [ ]:
joint = joint.dropna(subset = ['refDelay', 'refDep'])
joint.show()

+--------------+---------------+-----------+--------+-------+--------+------+
|year-month-day|OriginAirportID|OriginState|DepDelay|Deptime|refDelay|refDep|
+--------------+---------------+-----------+--------+-------+--------+------+
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|    76.0|1251.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|     1.0|1131.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|     1.0| 706.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|    16.0|1651.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|    15.0|1326.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|     0.0| 600.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|    57.0|1427.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|     1.0|1031.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|     1.0| 731.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|   

In [ ]:
pot_delay = joint.withColumn("diff", F.when((joint['Deptime'] > joint['refDep']+15) &  (joint['DepDelay'] > 0) , joint['DepDelay']).otherwise(0))
pot_delay.show()

+--------------+---------------+-----------+--------+-------+--------+------+----+
|year-month-day|OriginAirportID|OriginState|DepDelay|Deptime|refDelay|refDep|diff|
+--------------+---------------+-----------+--------+-------+--------+------+----+
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|    76.0|1251.0| 0.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|     1.0|1131.0| 0.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|     1.0| 706.0| 0.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|    16.0|1651.0| 0.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|    15.0|1326.0| 0.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|     0.0| 600.0| 0.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|    57.0|1427.0| 0.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|     1.0|1031.0| 0.0|
|     1995-1-13|          14492|         NC|    -1.0| 1014.0|     1.0| 731.0| 0.0|
|   

In [ ]:
final_r6 = pot_delay.groupBy('refDelay').agg(F.avg('diff'))#.orderBy('refDelay') final_r6 = final_r6.persist()
final_r6.show()

# REQUIREMENT 6 (Substitute):  the total distance travelled each year:
group by year and sum the distance

In [ ]:
df_dst = df.select('Year', 'Distance')
df_dst = df_dst.groupBy('Year').agg(F.sum('Distance').alias('TotalDistanceTravelled'))
df_dst.show()

+----+----------------------+
|Year|TotalDistanceTravelled|
+----+----------------------+
|1995|         3.636651803E9|
|1996|         3.717600021E9|
|1997|         3.844205684E9|
|1998|         3.866843234E9|
|1999|         4.043431545E9|
|2000|         4.197452626E9|
|2001|         4.207937834E9|
|2002|         3.952985686E9|
|2003|         4.549705964E9|
|2004|         5.004631442E9|
|2005|         5.080464469E9|
|2006|         5.116066945E9|
|2007|          6.16611809E8|
|2019|         5.834264894E9|
|2020|         3.438428349E9|
+----+----------------------+



In [ ]:
df_dst.toPandas().to_csv(output_dir+'req6/distance_year.csv')

# REQUIREMENT 7: The average delay across different seasons (eg, summer, winter) and which season is associated with the most delayed flights?

In [ ]:
mapping= {
        '1' : 'Winter',
        '2' : 'Winter',
        '3' : 'Spring',
        '4' : 'Spring',
        '5' : 'Spring',
        '6' : 'Summer',
        '7' : 'Summer',
        '8' : 'Summer',
        '9' : 'Fall',
        '10' : 'Fall',
        '11' : 'Fall',
        '12' : 'Winter'
    }
from pyspark.sql.types import StringType

seasons = df.select('Month', 'depDelay','arrDelay', 'Year').withColumn("Month",df["Month"].cast(StringType())).replace(to_replace=mapping, subset=['Month']).withColumnRenamed('Month','season').persist()
seasons.show()
final_r7 = seasons.groupBy('season', 'Year').agg(F.avg('depDelay'), F.avg('arrDelay'))
final_r7.show()


+------+--------+--------+----+
|season|depDelay|arrDelay|Year|
+------+--------+--------+----+
|Winter|    13.0|     8.0|1995|
|Winter|     2.0|     2.0|1995|
|Winter|    -1.0|     2.0|1995|
|Winter|     0.0|    -5.0|1995|
|Winter|    40.0|    43.0|1995|
|Winter|    12.0|    22.0|1995|
|Winter|    -3.0|   -19.0|1995|
|Winter|     0.0|    41.0|1995|
|Winter|    -1.0|   -11.0|1995|
|Winter|    -2.0|    -3.0|1995|
|Winter|    -3.0|    -7.0|1995|
|Winter|    14.0|     6.0|1995|
|Winter|    65.0|    50.0|1995|
|Winter|    -1.0|   -10.0|1995|
|Winter|    -1.0|    -6.0|1995|
|Winter|    -2.0|   -13.0|1995|
|Winter|    -2.0|   -19.0|1995|
|Winter|    12.0|     0.0|1995|
|Winter|    76.0|    61.0|1995|
|Winter|    -1.0|   -14.0|1995|
+------+--------+--------+----+
only showing top 20 rows

+------+----+------------------+------------------+
|season|Year|     avg(depDelay)|     avg(arrDelay)|
+------+----+------------------+------------------+
|Winter|1995|10.617912085773774| 9.699361366776284

In [ ]:
final_r7.toPandas().to_csv(output_dir+'req7/'+'season_year.csv')


In [ ]:
final_r7 = seasons.groupBy('season').agg(F.avg('depDelay'), F.avg('arrDelay'))
final_r7.show()

+------+------------------+------------------+
|season|     avg(depDelay)|     avg(arrDelay)|
+------+------------------+------------------+
|  Fall| 6.169798017473902|3.7735656254359498|
|Winter| 9.542297096456473| 7.573838154994453|
|Spring| 7.377447064681765| 5.154167888159128|
|Summer|10.073517201540762| 8.607588838223165|
+------+------------------+------------------+



In [ ]:
final_r7.toPandas().to_csv(output_dir+'req7/'+'seasonDelay.csv')


# REQUIREMENT 8: most popular destinations across years
>1- group by both year and destination city, count the number of flight records. 

>2- take a yearly window and find the city record associated with the maximum flight counts per year. 

In [ ]:
df8= df.select([ 'Year','DestCityName',
 'DestStateName'])

In [ ]:
df8 = df8.groupBy([ 'Year','DestCityName',
 'DestStateName']).count().withColumnRenamed('count', 'NumFlights').orderBy(F.col('NumFlights').desc() )

In [ ]:
df8.show()

+----+--------------------+-------------+----------+
|Year|        DestCityName|DestStateName|NumFlights|
+----+--------------------+-------------+----------+
|2006|         Chicago, IL|     Illinois|    458290|
|2004|         Chicago, IL|     Illinois|    453453|
|2003|         Chicago, IL|     Illinois|    440794|
|2005|         Chicago, IL|     Illinois|    420051|
|2005|         Atlanta, GA|      Georgia|    414237|
|2004|         Atlanta, GA|      Georgia|    407783|
|2019|         Chicago, IL|     Illinois|    407772|
|2006|         Atlanta, GA|      Georgia|    396738|
|2019|         Atlanta, GA|      Georgia|    391731|
|2002|         Chicago, IL|     Illinois|    371615|
|2001|         Chicago, IL|     Illinois|    369561|
|2003|         Atlanta, GA|      Georgia|    363093|
|2004|Dallas/Fort Worth...|        Texas|    336715|
|1995|         Chicago, IL|     Illinois|    332991|
|1997|         Chicago, IL|     Illinois|    331163|
|1999|         Chicago, IL|     Illinois|    3

In [ ]:
df8.count()

In [ ]:
from pyspark.sql import Window
w = Window.partitionBy('Year')
df8 = df8.withColumn('MaxNumFlights', F.max('NumFlights').over(w))\
    .where(F.col('NumFlights') == F.col('MaxNumFlights'))#\
    .drop('MaxNumFlights')

df8.show()


+----+------------+-------------+----------+-------------+
|Year|DestCityName|DestStateName|NumFlights|MaxNumFlights|
+----+------------+-------------+----------+-------------+
|1995| Chicago, IL|     Illinois|    332991|       332991|
|1996| Chicago, IL|     Illinois|    328765|       328765|
|1997| Chicago, IL|     Illinois|    331163|       331163|
|1998| Chicago, IL|     Illinois|    328882|       328882|
|1999| Chicago, IL|     Illinois|    330125|       330125|
|2000| Chicago, IL|     Illinois|    327665|       327665|
|2001| Chicago, IL|     Illinois|    369561|       369561|
|2002| Chicago, IL|     Illinois|    371615|       371615|
|2003| Chicago, IL|     Illinois|    440794|       440794|
|2004| Chicago, IL|     Illinois|    453453|       453453|
|2005| Chicago, IL|     Illinois|    420051|       420051|
|2006| Chicago, IL|     Illinois|    458290|       458290|
|2007| Atlanta, GA|      Georgia|     57751|        57751|
|2019| Chicago, IL|     Illinois|    407772|       40777

In [ ]:
df8.toPandas().to_csv(output_dir+'req8/most_popular_dest.csv')

# REQUIREMENT 9: busiest airports across years
1- df1:group by both year and originairportid and count the flight records.

2- df2:group by both year and destairportid and count the flight records.

3- join the two dfs on airport id and sum the two flights count columns.

In [ ]:
df_orig= df.select([ 'Year','OriginAirportID',
 'Origin',
 'OriginCityName',
 'OriginStateName'])

df_dest = df.select(['Year', 'DestAirportID',
 'Dest',
 'DestCityName',
 'DestStateName'])



In [ ]:
df_orig =df_orig.groupBy('Year','OriginAirportID').agg(F.count(F.lit(1)).alias("NumFlights"),\
                                                 F.collect_set('Origin').alias('Airport'),  \
                                                 F.collect_set('OriginCityName').alias('City'),\
                                                 F.collect_set('OriginStateName').alias('State')).withColumnRenamed('OriginAirportID', 'AirportID')
df_orig.show()

+----+---------+----------+-------+--------------------+---------------+
|Year|AirportID|NumFlights|Airport|                City|          State|
+----+---------+----------+-------+--------------------+---------------+
|1995|    10135|      6963|  [ABE]|[Allentown/Bethle...| [Pennsylvania]|
|1995|    10140|     36591|  [ABQ]|   [Albuquerque, NM]|   [New Mexico]|
|1995|    10154|        38|  [ACK]|     [Nantucket, MA]|[Massachusetts]|
|1995|    10170|       463|  [ADQ]|        [Kodiak, AK]|       [Alaska]|
|1995|    10208|      2347|  [AGS]|       [Augusta, GA]|      [Georgia]|
|1995|    10257|     11705|  [ALB]|        [Albany, NY]|     [New York]|
|1995|    10279|      4254|  [AMA]|      [Amarillo, TX]|        [Texas]|
|1995|    10469|      2390|  [AZO]|     [Kalamazoo, MI]|     [Michigan]|
|1995|    10529|     25231|  [BDL]|      [Hartford, CT]|  [Connecticut]|
|1995|    10551|       941|  [BET]|        [Bethel, AK]|       [Alaska]|
|1995|    10577|      1372|  [BGM]|    [Binghamton,

In [ ]:
df_dest =df_dest.groupBy('Year','DestAirportID').agg(F.count(F.lit(1)).alias("NumFlights2")).withColumnRenamed('DestAirportID', 'AirportID')
df_dest.show()

+----+---------+-----------+
|Year|AirportID|NumFlights2|
+----+---------+-----------+
|1995|    15412|       9327|
|1995|    10279|       4244|
|1995|    10821|      65443|
|1995|    12441|       1388|
|1995|    13476|       1165|
|1995|    14986|       6790|
|1995|    14107|     154913|
|1995|    13029|       3410|
|1995|    14122|     123414|
|1995|    14576|      15800|
|1995|    15024|       2800|
|1995|    10170|        463|
|1995|    10577|       1377|
|1995|    12339|      36546|
|1995|    11337|       1715|
|1995|    11641|       2158|
|1995|    12191|      56167|
|1995|    12523|       4021|
|1995|    13930|     286048|
|1995|    13184|       3721|
+----+---------+-----------+
only showing top 20 rows



In [ ]:
req9_df = df_orig.join(df_dest, ['Year', 'AirportID'])
req9_df.show()

+----+---------+----------+-------+--------------------+--------------------+-----------+
|Year|AirportID|NumFlights|Airport|                City|               State|NumFlights2|
+----+---------+----------+-------+--------------------+--------------------+-----------+
|1995|    15412|      9313|  [TYS]|     [Knoxville, TN]|         [Tennessee]|       9327|
|1995|    10279|      4254|  [AMA]|      [Amarillo, TX]|             [Texas]|       4244|
|1995|    10821|     65129|  [BWI]|     [Baltimore, MD]|          [Maryland]|      65443|
|1995|    12441|      1391|  [JAC]|       [Jackson, WY]|           [Wyoming]|       1388|
|1995|    13476|      1162|  [MRY]|      [Monterey, CA]|        [California]|       1165|
|1995|    14986|      6777|  [SRQ]|[Sarasota/Bradent...|           [Florida]|       6790|
|1995|    14107|    155207|  [PHX]|       [Phoenix, AZ]|           [Arizona]|     154913|
|1995|    13029|      3400|  [LNK]|       [Lincoln, NE]|          [Nebraska]|       3410|
|1995|    

In [ ]:
req9_df = req9_df.withColumn('NumFlights', req9_df.NumFlights + req9_df.NumFlights2).drop('NumFlights2')
req9_df.show()

+----+---------+----------+-------+--------------------+--------------------+
|Year|AirportID|NumFlights|Airport|                City|               State|
+----+---------+----------+-------+--------------------+--------------------+
|1995|    15412|     18640|  [TYS]|     [Knoxville, TN]|         [Tennessee]|
|1995|    10279|      8498|  [AMA]|      [Amarillo, TX]|             [Texas]|
|1995|    10821|    130572|  [BWI]|     [Baltimore, MD]|          [Maryland]|
|1995|    12441|      2779|  [JAC]|       [Jackson, WY]|           [Wyoming]|
|1995|    13476|      2327|  [MRY]|      [Monterey, CA]|        [California]|
|1995|    14986|     13567|  [SRQ]|[Sarasota/Bradent...|           [Florida]|
|1995|    14107|    310120|  [PHX]|       [Phoenix, AZ]|           [Arizona]|
|1995|    13029|      6810|  [LNK]|       [Lincoln, NE]|          [Nebraska]|
|1995|    14122|    247189|  [PIT]|    [Pittsburgh, PA]|      [Pennsylvania]|
|1995|    14576|     31596|  [ROC]|     [Rochester, NY]|        

In [ ]:
from pyspark.sql import Window
w = Window.partitionBy('Year')
req9_df = req9_df.withColumn('MaxNumFlights', F.max('NumFlights').over(w))\
    .where(F.col('NumFlights') == F.col('MaxNumFlights'))\
    .drop('MaxNumFlights')

req9_df.show()


+----+---------+----------+-------+-------------+----------+
|Year|AirportID|NumFlights|Airport|         City|     State|
+----+---------+----------+-------+-------------+----------+
|1995|    13930|    572749|  [ORD]|[Chicago, IL]|[Illinois]|
|1996|    13930|    571100|  [ORD]|[Chicago, IL]|[Illinois]|
|1997|    13930|    576583|  [ORD]|[Chicago, IL]|[Illinois]|
|1998|    13930|    568647|  [ORD]|[Chicago, IL]|[Illinois]|
|1999|    13930|    562289|  [ORD]|[Chicago, IL]|[Illinois]|
|2000|    13930|    552341|  [ORD]|[Chicago, IL]|[Illinois]|
|2001|    13930|    642088|  [ORD]|[Chicago, IL]|[Illinois]|
|2002|    13930|    640172|  [ORD]|[Chicago, IL]|[Illinois]|
|2003|    10397|    726946|  [ATL]|[Atlanta, GA]| [Georgia]|
|2004|    10397|    817248|  [ATL]|[Atlanta, GA]| [Georgia]|
|2005|    10397|    833284|  [ATL]|[Atlanta, GA]| [Georgia]|
|2006|    10397|    797170|  [ATL]|[Atlanta, GA]| [Georgia]|
|2007|    10397|    115782|  [ATL]|[Atlanta, GA]| [Georgia]|
|2019|    10397|    7834

In [ ]:
req9_df.toPandas().to_csv(output_dir+'req9/busiest_airport.csv')

# REQUIREMENT 10: ML requirement- delay prediction regression model

In [ ]:
cols = [ 'Month',
 'DayofMonth',
 'DayOfWeek',
 'OriginAirportID',
 'Origin',
 'OriginState',
 'DestAirportID',
 'Dest',
 'DestState',
 'DepDelay',
 'ArrTime',
 'Distance']
import six
for i in df.columns:
    if not( isinstance(df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to dep delay for ", i, df.stat.corr('DepDelay',i))

Correlation to dep delay for  Year -0.013171289725626095
Correlation to dep delay for  Month -0.003671084688269536
Correlation to dep delay for  DayofMonth 0.006122770761440702
Correlation to dep delay for  DayOfWeek 0.01099746638637157
Correlation to dep delay for  Flight_Number_Reporting_Airline 0.005514420612268031
Correlation to dep delay for  OriginAirportID -0.0035020016265860368
Correlation to dep delay for  DestAirportID 0.010188892936520512
Correlation to dep delay for  DepTime 0.16357292463144735
Correlation to dep delay for  DepDelay 1.0
Correlation to dep delay for  ArrTime 0.06239139748473187
Correlation to dep delay for  ArrDelay 0.8962735253179147
Correlation to dep delay for  Cancelled nan
Correlation to dep delay for  Diverted nan
Correlation to dep delay for  ActualElapsedTime 0.024816438882012175
Correlation to dep delay for  Flights nan
Correlation to dep delay for  Distance 0.013940265958648826


**based on these correlation values, it doesn't look like  this is going to work out**

In [ ]:
features_list = ['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'OriginAirportID',
 'DestAirportID',
 'Distance',
 'DepDelay']
reg_df=df.select(features_list)
reg_df.show()

+----+-----+----------+---------+---------------+-------------+--------+--------+
|Year|Month|DayofMonth|DayOfWeek|OriginAirportID|DestAirportID|Distance|DepDelay|
+----+-----+----------+---------+---------------+-------------+--------+--------+
|1995|    1|        26|        4|          13303|        15024|  1107.0|    13.0|
|1995|    1|        27|        5|          13303|        15024|  1107.0|     2.0|
|1995|    1|        28|        6|          13303|        15024|  1107.0|    -1.0|
|1995|    1|        29|        7|          13303|        15024|  1107.0|     0.0|
|1995|    1|        30|        1|          13303|        15024|  1107.0|    40.0|
|1995|    1|        31|        2|          13303|        15024|  1107.0|    12.0|
|1995|    1|         1|        7|          13303|        15024|  1107.0|    -3.0|
|1995|    1|         2|        1|          13303|        15024|  1107.0|     0.0|
|1995|    1|         3|        2|          13303|        15024|  1107.0|    -1.0|
|1995|    1|    

In [ ]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols =features_list[:-1] , outputCol = 'Features')
reg_df = vectorAssembler.transform(reg_df)
dataset= reg_df.select(['Features', 'DepDelay']).withColumnRenamed('DepDelay', 'y')
dataset.show(3)



+--------------------+----+
|            Features|   y|
+--------------------+----+
|[1995.0,1.0,26.0,...|13.0|
|[1995.0,1.0,27.0,...| 2.0|
|[1995.0,1.0,28.0,...|-1.0|
+--------------------+----+
only showing top 3 rows



In [ ]:
splits = dataset.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [ ]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'Features', labelCol='y', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
#print("Coefficients: " + str(lr_model.coefficients))
#print("Intercept: " + str(lr_model.intercept))

Coefficients: [-0.02623651728560547,0.0,0.0,0.05697991352939514,0.0,4.638437521639852e-05,0.0003679850162082815]
Intercept: 59.7816002885579


In [ ]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 32.234324
r2: 0.000364


In [ ]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","y","Features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="y",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+-----------------+----+--------------------+
|       prediction|   y|            Features|
+-----------------+----+--------------------+
|8.575511678821776|-5.0|[1995.0,1.0,1.0,7...|
|8.575511678821776| 7.0|[1995.0,1.0,1.0,7...|
|8.417296419618744|-3.0|[1995.0,1.0,1.0,7...|
|8.725404245845404| 0.0|[1995.0,1.0,1.0,7...|
|8.725404245845404| 3.0|[1995.0,1.0,1.0,7...|
+-----------------+----+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.000360826


**and yup, it doesn't work out :"D**

## okay let's try sth else. let's see if we can predict delay cancellation

In [ ]:
df_cancel=spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('/content/drive/MyDrive/BigData-finalproject/complete_data/cancellation.csv').persist()
df_cancel.show()

+----+-----+----------+---------+---------------+------------------+---------------+-------------+------------+-------------+---------+
|Year|Month|DayofMonth|DayOfWeek|OriginAirportID|    OriginCityName|OriginStateName|DestAirportID|DestCityName|DestStateName|Cancelled|
+----+-----+----------+---------+---------------+------------------+---------------+-------------+------------+-------------+---------+
|1996|    1|        24|        3|          14869|Salt Lake City, UT|           Utah|        11292|  Denver, CO|     Colorado|      0.0|
|1996|    1|        25|        4|          14869|Salt Lake City, UT|           Utah|        11292|  Denver, CO|     Colorado|      0.0|
|1996|    1|        26|        5|          14869|Salt Lake City, UT|           Utah|        11292|  Denver, CO|     Colorado|      0.0|
|1996|    1|        27|        6|          14869|Salt Lake City, UT|           Utah|        11292|  Denver, CO|     Colorado|      0.0|
|1996|    1|        29|        1|          14869

In [ ]:
df_complete = df.select(df_cancel.columns).union(df_cancel)
df_complete.count()

162197331

In [ ]:
import six
for i in df_complete.columns:
    if not( isinstance(df_complete.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to cancellation for ", i, df_complete.stat.corr('Cancelled',i))

Correlation to cancellation for  Year 0.017612986120476
Correlation to cancellation for  Month -0.01912255865091238
Correlation to cancellation for  DayofMonth 0.001452778081725812
Correlation to cancellation for  DayOfWeek -0.00854110535381647
Correlation to cancellation for  OriginAirportID -0.00526472231961758
Correlation to cancellation for  DestAirportID -0.006049256889372834
Correlation to cancellation for  Cancelled 1.0


outch
😂

In [ ]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols =['Year','Month','DayofMonth', 'DayOfWeek', 'OriginAirportID','DestAirportID'], outputCol = 'Features')
can_df = vectorAssembler.transform(df_complete)
dataset= can_df.select(['Features', 'Cancelled']).withColumnRenamed('Cancelled', 'y')
dataset.show(3)



+--------------------+---+
|            Features|  y|
+--------------------+---+
|[1995.0,1.0,26.0,...|0.0|
|[1995.0,1.0,27.0,...|0.0|
|[1995.0,1.0,28.0,...|0.0|
+--------------------+---+
only showing top 3 rows



In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler

train, test = dataset.randomSplit([0.7, 0.3], seed = 2000)
#print("Training Dataset Count: " + str(train.count()))
#print("Test Dataset Count: " + str(test.count()))

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'Features', labelCol = 'y', maxDepth = 2)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)
predictions.show(10)

+--------------------+---+--------------------+--------------------+----------+
|            Features|  y|       rawPrediction|         probability|prediction|
+--------------------+---+--------------------+--------------------+----------+
|[1995.0,1.0,1.0,7...|0.0|[1.12192304E8,135...|[0.98810763474366...|       0.0|
|[1995.0,1.0,1.0,7...|0.0|[1.12192304E8,135...|[0.98810763474366...|       0.0|
|[1995.0,1.0,1.0,7...|0.0|[1.12192304E8,135...|[0.98810763474366...|       0.0|
|[1995.0,1.0,1.0,7...|0.0|[1.12192304E8,135...|[0.98810763474366...|       0.0|
|[1995.0,1.0,1.0,7...|0.0|[1.12192304E8,135...|[0.98810763474366...|       0.0|
|[1995.0,1.0,1.0,7...|0.0|[1.12192304E8,135...|[0.98810763474366...|       0.0|
|[1995.0,1.0,1.0,7...|0.0|[1.12192304E8,135...|[0.98810763474366...|       0.0|
|[1995.0,1.0,1.0,7...|0.0|[1.12192304E8,135...|[0.98810763474366...|       0.0|
|[1995.0,1.0,1.0,7...|0.0|[1.12192304E8,135...|[0.98810763474366...|       0.0|
|[1995.0,1.0,1.0,7...|0.0|[1.12192304E8,

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print('Test Area Under ROC', evaluator.evaluate(predictions.withColumnRenamed('y', 'label')))

Test Area Under ROC 0.5
